In [1]:
from textblob import TextBlob

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report

In [3]:
dev_data = np.load("/Users/mehradghassemi/twitter_sentiment/data/interim/dev_data.npy", allow_pickle=True)

In [4]:
dev_data[0]

array([567729749087498240, 'negative', 1.0, 'Lost Luggage', 1.0,
       'US Airways', nan, 'itsjustdoc', nan, 0,
       "@USAirways I didn't even leave the airport and you sent 2 of my bags to Philadelphia!",
       nan, '2015-02-17 08:58:14 -0800', 'Ohio', 'Quito'], dtype=object)

In [5]:
df = pd.read_csv("/Users/mehradghassemi/twitter_sentiment/data/raw/Tweets.csv")

In [6]:
columns = df.columns

In [7]:
columns

Index(['tweet_id', 'airline_sentiment', 'airline_sentiment_confidence',
       'negativereason', 'negativereason_confidence', 'airline',
       'airline_sentiment_gold', 'name', 'negativereason_gold',
       'retweet_count', 'text', 'tweet_coord', 'tweet_created',
       'tweet_location', 'user_timezone'],
      dtype='object')

In [8]:
dev_df = pd.DataFrame(dev_data, columns=columns)

In [9]:
dev_df

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567729749087498240,negative,1.0,Lost Luggage,1.0,US Airways,NaN,itsjustdoc,NaN,0,@USAirways I didn't even leave the airport and...,NaN,2015-02-17 08:58:14 -0800,Ohio,Quito
1,569591653121597440,negative,1.0,Customer Service Issue,0.7255,American,NaN,kiabeveridge,NaN,0,"@AmericanAir I used the ""call back"" feature wi...",NaN,2015-02-22 12:16:47 -0800,Chicago,Mountain Time (US & Canada)
2,569599867716132864,positive,1.0,NaN,NaN,Virgin America,NaN,jessicajaymes,NaN,2,Always have it together!!! You're welcome! RT ...,"[33.94652852, -118.40766257]",2015-02-22 12:49:25 -0800,"hollywood, california",Pacific Time (US & Canada)
3,569630837538361344,positive,0.6775,NaN,NaN,United,NaN,achoifitz,NaN,0,"@united finally made it to rep, who solved my ...",NaN,2015-02-22 14:52:29 -0800,Europe - America,Central Time (US & Canada)
4,568245558919503872,negative,1.0,Customer Service Issue,1.0,US Airways,NaN,bhardwaj_j,NaN,0,@USAirways : its just a very bad customer serv...,NaN,2015-02-18 19:07:53 -0800,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1459,568450643175866368,negative,0.7021,Customer Service Issue,0.7021,Southwest,NaN,TMP8622,NaN,0,@SouthwestAir I already follow you and can't D...,NaN,2015-02-19 08:42:49 -0800,"Tempe, AZ",Arizona
1460,568260084230455296,negative,1.0,Customer Service Issue,0.6848,United,NaN,Belendelcanto,NaN,0,@united its not the way to treat your customer...,NaN,2015-02-18 20:05:36 -0800,Seattle,Hawaii
1461,569964335038124033,negative,1.0,Customer Service Issue,0.657,American,NaN,aepleiss,NaN,0,@AmericanAir I don't believe it's acceptable t...,NaN,2015-02-23 12:57:41 -0800,NaN,NaN
1462,569208236487745536,negative,0.6765,Customer Service Issue,0.6765,Southwest,NaN,jaretgordon,NaN,0,@SouthwestAir have a prompt for consumers for ...,NaN,2015-02-21 10:53:13 -0800,NaN,Central Time (US & Canada)


In [10]:
dev_df.shape

(1464, 15)

In [11]:
predicted_polarity = dev_df.text.apply(lambda x: TextBlob(x).sentiment[0])

In [12]:
predicted_polarity

0       0.000
1      -0.150
2       0.900
3       0.000
4      -0.910
        ...  
1459    0.000
1460   -0.150
1461    0.025
1462    0.000
1463    0.000
Name: text, Length: 1464, dtype: float64

In [13]:
dev_df.airline_sentiment.value_counts()

airline_sentiment
negative    938
neutral     305
positive    221
Name: count, dtype: int64

In [14]:
predicted_polarity.describe()

count    1464.000000
mean        0.040315
std         0.323940
min        -1.000000
25%        -0.050000
50%         0.000000
75%         0.200000
max         1.000000
Name: text, dtype: float64

In [15]:
positive_cutoff = 0.2
negative_cutoff = -0.05

In [16]:
def convert_polarity(polarity_score): 
    if polarity_score >= positive_cutoff:
        return 2
    elif polarity_score <= negative_cutoff:
        return 1
    else:
        return 0
    
predicted_class = predicted_polarity.apply(lambda x: convert_polarity(x))

In [17]:
def convert_sentiment(category):
    if category == "positive":
        return 2
    elif category =="neutral":
        return 1
    else:
        return 0
    
actual_class = dev_df.airline_sentiment.apply(lambda x: convert_sentiment(x))

In [18]:
print(classification_report(actual_class, predicted_class))

              precision    recall  f1-score   support

           0       0.61      0.48      0.54       938
           1       0.08      0.09      0.08       305
           2       0.39      0.65      0.49       221

    accuracy                           0.42      1464
   macro avg       0.36      0.41      0.37      1464
weighted avg       0.47      0.42      0.44      1464

